In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
pip install pytictoc

In [3]:
import numpy as np
import keras
import sklearn as sk
import cv2
from keras.applications.resnet import ResNet50
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input, Dense, Flatten
import pytictoc
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import glob
import os

Using TensorFlow backend.


In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
time_print = pytictoc.TicToc()
time_print.tic()
videos = []
labels_2d = []
num_frames = 100
frame_rate = 5
time_3d = []
time_2d = []
############################# Videos with label = 1
i=0
for path in glob.glob('/content/drive/My Drive/CSCE636/9_training_2/1/*.mp4'):
  
  vidcap = cv2.VideoCapture(path)
  fps = vidcap.get(cv2.CAP_PROP_FPS)
  success, image = vidcap.read()
  frames = []
  

  time = []
  count = 0  # control to have the same number of frames
  count_fps = 0
  while success:
    
    
    success, image = vidcap.read()
    count += 1
    if(type(image).__module__ == np.__name__):
      new_image = cv2.resize(image, (224,224), interpolation = cv2.INTER_AREA)
      
      frames.append(new_image)
      time.append(count_fps*(1/fps))
      count_fps += 1
      if count==num_frames:
        count_fps = 0
        print("Frames_", str(count),", video_", str(i), "Done, Time_Elapsed: ", str(round(time_print.tocvalue(),3)), " Seconds")
        videos.append(frames)
        time_3d.append(time)
        count = 0
        frames = []
        time = []

  if (count < num_frames):
    while (count > 0 and count <= num_frames):
      frames.append(new_image)    # if the number of frames is lower than the num_frames, repeat the last image to reach num_frames
      count +=1
      time.append(count_fps*(1/fps))
      count_fps += 1
      
    videos.append(frames)
    time_3d.append(time)
  
  i+=1
  print("Video_", str(i),"Done, Time_Elapsed: ", str(round(time_print.tocvalue(),3)), " Seconds")


videos_2d_len = len(videos)

############################# Videos with label = 0
i=0
for path in glob.glob('/content/drive/My Drive/CSCE636/9_training_2/0/*.mp4'):
  if(i == 219):
    break
  
  vidcap = cv2.VideoCapture(path)
  fps = vidcap.get(cv2.CAP_PROP_FPS)
  success, image = vidcap.read()
  frames = []
  
  time = []
  count = 0  # control to have the same number of frames
  count_fps = 0
  while success:
    
    success, image = vidcap.read()
    count += 1
    if(type(image).__module__ == np.__name__):
      new_image = cv2.resize(image, (224,224), interpolation = cv2.INTER_AREA)
    
      frames.append(new_image)
      time.append(count_fps*(1/fps))
      count_fps += 1
      if count==num_frames:
        count_fps = 0
        print("Frames_", str(count),", video_", str(i), "Done, Time_Elapsed: ", str(round(time_print.tocvalue(),3)), " Seconds")
        videos.append(frames)
        time_3d.append(time)
        count = 0
        frames = []
        time = []


  if (count < num_frames):
    while (count > 0 and count <= num_frames):
      frames.append(new_image)    # if the number of frames is lower than the num_frames, repeat the last image to reach num_frames
      count +=1
      time.append(count_fps*(1/fps))
      count_fps += 1
      
    videos.append(frames)
    time_3d.append(time)

  i+=1
  print("Video_", str(i),"Done, Time_Elapsed: ", str(round(time_print.tocvalue(),3)), " Seconds")

videos_3d = np.array(videos)

labels_2d = np.zeros((len(videos_3d),num_frames))
labels_2d[0:videos_2d_len,:] = 1

time_3d = np.array(time_3d)

Video_ 1 Done, Time_Elapsed:  2.969  Seconds
Video_ 2 Done, Time_Elapsed:  4.468  Seconds
Video_ 3 Done, Time_Elapsed:  6.177  Seconds
Video_ 4 Done, Time_Elapsed:  7.583  Seconds
Video_ 5 Done, Time_Elapsed:  9.341  Seconds
Video_ 6 Done, Time_Elapsed:  10.832  Seconds
Video_ 7 Done, Time_Elapsed:  12.311  Seconds
Video_ 8 Done, Time_Elapsed:  13.667  Seconds
Video_ 9 Done, Time_Elapsed:  15.097  Seconds
Video_ 10 Done, Time_Elapsed:  16.241  Seconds
Video_ 11 Done, Time_Elapsed:  17.253  Seconds
Video_ 12 Done, Time_Elapsed:  17.991  Seconds
Video_ 13 Done, Time_Elapsed:  18.644  Seconds
Video_ 14 Done, Time_Elapsed:  19.895  Seconds
Video_ 15 Done, Time_Elapsed:  21.44  Seconds
Video_ 16 Done, Time_Elapsed:  22.916  Seconds
Video_ 17 Done, Time_Elapsed:  24.592  Seconds
Video_ 18 Done, Time_Elapsed:  26.23  Seconds
Video_ 19 Done, Time_Elapsed:  28.523  Seconds
Video_ 20 Done, Time_Elapsed:  29.311  Seconds
Video_ 21 Done, Time_Elapsed:  29.956  Seconds
Video_ 22 Done, Time_Elapsed:

All videos will be converted to frames and saved as numpy files




In [6]:
#Correcting shapes of images
ind = list(np.random.randint(0,len(videos_3d)-1,size=len(videos_3d)))
videos_3d_temp = videos_3d[ind]
videos_2d_shuffled = np.reshape(videos_3d_temp, (-1,224,224,3))
videos_2d_shuffled = videos_2d_shuffled[:len(videos_2d_shuffled):frame_rate]

time_3d_temp = time_3d[ind]
time_2d_shuffled = np.reshape(time_3d_temp, (-1,1))
time_2d_shuffled = time_2d_shuffled[:len(time_2d_shuffled):frame_rate]
time_2d_shuffled = time_2d_shuffled.T[0]

labels_2d_temp = labels_2d[ind]
labels_2d_shuffled = labels_2d_temp.flatten()
labels_2d_shuffled = labels_2d_shuffled[:len(labels_2d_shuffled):frame_rate]

In [7]:
#Saving all the vidoes as npy formats to reduce time for reading video files
np.save("/content/drive/My Drive/CSCE636/9_Dataset/raw_data.npy", videos_2d_shuffled)
np.save("/content/drive/My Drive/CSCE636/9_Dataset/raw_label.npy", labels_2d_shuffled)
np.save("/content/drive/My Drive/CSCE636/9_Dataset/times.npy", time_2d_shuffled)

In [ ]:
pip install pytictoc

In [ ]:
np.shape(time_3d)

(295, 100)